In [1]:
import os
os.getpid()

21186

In [2]:
import json
f = open('ESConv.json')
data = json.load(f)
f.close()
processed_data = []
for i in data:
    pt = i['problem_type']
    for j in i['dialog']:
        processed_data.append([pt, j['content'], j['speaker'], j['annotation']])

In [3]:
processed_data[4005]

['ongoing depression',
 'Sometimes we get in a funk, when we do, we can pull ourselves out, I do suggest you write a list of things to do and put on the frige.',
 'supporter',
 {'strategy': 'Others'}]

In [4]:
sl = []
for i in processed_data:
    sl.append(len(i[1].split()))
## Safely, we remove length 1,2,3
to_remove = []
for i in range(len(sl)):
    if sl[i] <= 3:
        to_remove.append(i)
for index in sorted(to_remove, reverse=True):
    del processed_data[index]

In [5]:
from collections import Counter
cpd = []
for i in processed_data:
    cpd.append(i[0])
Counter(cpd)

Counter({'job crisis': 7475,
         'problems with friends': 4702,
         'ongoing depression': 9462,
         'breakup with partner': 6697,
         'academic pressure': 4009,
         'conflict with parents': 255,
         'Procrastination': 352,
         'Alcohol Abuse': 360,
         'Issues with Parents': 239,
         'Sleep Problems': 765,
         'Appearance Anxiety': 346,
         'School Bullying': 46,
         'Issues with Children': 371})

In [6]:
# MERGE Issue with parents
for i in range(len(processed_data)):
    if processed_data[i][0] == 'conflict with parents':
        processed_data[i][0] = 'Issues with Parents'
# MERGE Into Family Issue
for i in range(len(processed_data)):
    if processed_data[i][0] == 'Issues with Parents':
        processed_data[i][0] = 'Family Issues'
    if processed_data[i][0] == 'Issues with Children':
        processed_data[i][0] = 'Family Issues'

In [7]:
cpd = []
for i in processed_data:
    cpd.append(i[0])
Counter(cpd)

Counter({'job crisis': 7475,
         'problems with friends': 4702,
         'ongoing depression': 9462,
         'breakup with partner': 6697,
         'academic pressure': 4009,
         'Family Issues': 865,
         'Procrastination': 352,
         'Alcohol Abuse': 360,
         'Sleep Problems': 765,
         'Appearance Anxiety': 346,
         'School Bullying': 46})

In [8]:
# For now, we care the all 7 categories.
kept = ['problems with friends', 'ongoing depression','breakup with partner', 'academic pressure', 'job crisis', 'Family Issues','Sleep Problems', 'illness']

filtered_processed_data = []
for i in processed_data:
    if i[0] in kept:
        filtered_processed_data.append(list(i))
        filtered_processed_data[-1][0] = kept.index(filtered_processed_data[-1][0])

In [9]:
import random

random.seed(10)
random.shuffle(filtered_processed_data)
lenth = len(filtered_processed_data)
train_list = filtered_processed_data[0:int(lenth*0.8)]
test_list = filtered_processed_data[int(lenth*0.8):]

In [10]:
# Count labels in training
lt = []
for i in train_list:
    lt.append(i[0])
Counter(lt)

Counter({2: 5364, 1: 7583, 6: 628, 4: 5986, 3: 3144, 0: 3767, 5: 708})

In [11]:
# Count labels in test
lt = []
for i in test_list:
    lt.append(i[0])
Counter(lt)

Counter({2: 1333, 1: 1879, 3: 865, 4: 1489, 0: 935, 5: 157, 6: 137})

# Now we modify the training set, force a balanced amount of data across all classes

In [12]:
classes = [[],[],[],[],[],[],[]]

for i in train_list:
    classes[i[0]].append(i)

In [13]:
sl = [[],[],[],[],[],[],[]]
for i in range(7):
    for j in range(len(classes[i])):
        sl[i].append(len(classes[i][j][1].split()))
for i in sl:
    i.sort()

In [14]:
for i in range(7):
    random.shuffle(classes[i])
    classes[i] = classes[i][:627]

In [15]:
train_list = []
for i in classes:
    train_list.extend(i)
random.shuffle(train_list)

In [16]:
import csv
import pandas as pd
patient_data = pd.read_csv('patient_notes.csv')
i = 0
cap = 627
while i < cap:
    i += 1
    try:
        illness_tr = patient_data.iloc[i*10]['pn_history'].split('. ')[2]
        illness_te = patient_data.iloc[i*10]['pn_history'].split('. ')[2]
        if illness_tr[:2] == 'He':
            illness_tr = 'I'+ illness_tr[2:]
        if illness_tr[:3] == 'She':
            illness_tr = 'I'+ illness_tr[3:]
        if illness_te[:2] == 'He':
            illness_te = 'I'+ illness_tr[2:]
        if illness_te[:3] == 'She':
            illness_te = 'I'+ illness_tr[3:]            
        train_list.append([7,illness_tr ])
        test_list.append([7, illness_te])
    except:
        cap += 1
        

In [42]:
for i in train_list:
    if i[0] == 7:
        print(i[1])

The patient reports that began experiencing palpitations/heart racing about two months ago
Most episodes have occurred at rest although the latest episode occurred during a soccer game
Pt reports feeling like he was going to lose consciousness, however he did not, no diaphoresis
IT STARTED SUDDENLY
THE PT DENIED, DIZZINESS, HEADCHE, COUGH, GIT OR URINARY SYMPTOMS, WT LOSS OR APPPETITE CHANGES , ANY CHANGES IN APPETITE OR W\DIET , ANY TEPRETURE INTOLRANCE
ROS: -VE EXCPT AABOVE
NKA
MED: ADDERAL THE LAST DOSE 2 DAYS BACK
FH: MOTHER W/ THYROID PROBLEM 
SH; DRINKING ETOH OCCASIONLY AT THE WEEK ENDS, NO ETOH, NO OTHER ILLICIT DRUGS
It happens in any setting and they are unpredictable
I has experienced about 5-6 episodes so far
I reports 5-6 episodes of palpitations ("heart pounding") each lasting 3-4 minutes, with the last episode accompanied by mid-line, non-radiating chest pressure and dyspnea

He denies any syncope, headaches, vision changes, cough, weakness, paresthesias, heat intoleranc

In [17]:
# Count labels in training
lt = []
for i in train_list:
    lt.append(i[0])
Counter(lt)

Counter({2: 627, 4: 627, 1: 627, 3: 627, 5: 627, 0: 627, 6: 627, 7: 627})

In [18]:
# Count labels in training
lt = []
for i in test_list:
    lt.append(i[0])
Counter(lt)

Counter({2: 1333, 1: 1879, 3: 865, 4: 1489, 0: 935, 5: 157, 6: 137, 7: 627})

In [19]:
import torch
from torch.utils.data.sampler import BatchSampler, Sampler, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel, AdamW, BertConfig
from tqdm import tqdm

class Corpora():
    """
        A class that holds the texts and class labels.
    """
    
    def __init__(self, l):
        """
            file_path: on MAGIC, it is either
                        data-badassnlp/cola_public/raw/in_domain_train.tsv or
                        data-badassnlp/cola_public/raw/in_domain_dev.tsv
                    on Google Colab, change them to the corresponding paths.
        """

        self.sentences = [s[1] for s in tqdm(l)]
        self.labels = [int(s[0]) for s in tqdm(l)]
        
        
class TextClassificationDataSet(Dataset):
    """
        Define a dataset consisting of pairs of (sequence_of_word_indices, class_label).
        class_label is either 0 or 1.
    """
    def __init__(self, corpora, tokenizer: BertTokenizer):
        """
            This function will tokenize the sentences in self.corpora,
            and pad the word indices properly.
            The tokenization and padding should be done using BERT's API.

                See https://huggingface.co/docs/transformers/internal/tokenization_utils
                    for the API of tokenizer
                See https://huggingface.co/docs/transformers/preprocessing
                    for examples of using the API
                Also see the BERT paper
                    "BERT: Pre-training of Deep Bidirectional Transformers forLanguage Understanding"
                regarding how paddings are done.

            We turn the entire corpora into a big tensor in a batch, rather than
                doing that in the __getitem__ function, as batch processing can speed up tokenization
        
            Then self.input_ids, self.attention_mask and self.labels
            will be created as tensors of shape
                torch.Size([num_sentences, max_padded_sentence_length])
                torch.Size([num_sentences, max_padded_sentence_length])
                torch.Size([num_sentences])

            corpora: an object of the Corpora class representing some raw classified texts.
            tokenizer: must be the BERTTokenizer loaded from the BERT/download folder
                        in order to have the same ids for the words in any vocabulary.
                        See the loading codes below.
        """
        self.corpora = corpora
        self.tokenizer = tokenizer
        d =  tokenizer.batch_encode_plus(corpora.sentences)
        padded_encoded_inputs = tokenizer.pad(d)
        self.input_ids = torch.tensor(padded_encoded_inputs.input_ids)
        self.attention_mask = torch.tensor(padded_encoded_inputs.attention_mask)
        self.labels = torch.tensor(corpora.labels)
        
    def __len__(self):
        return len(self.corpora.sentences)

    def __getitem__(self, idx):
        """
            Return the idx-th of the rows the self.input_ids, self.attention_mask, self.labels in this order.
            Don't do BERT tokenization here as that will be slow.
        """
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [20]:
from torch import nn

class BERTClassifier(nn.Module):
    
    def __init__(self, BERT_model, hidden_layer_size, num_classes):
        
        super(BERTClassifier, self).__init__()

        # loaded pretrained model
        self.bert = BERT_model
        
        # simple neural network that convert embedding of the first token to a class
        self.classifier = nn.Sequential(
            nn.Linear(BERT_hidden_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        """
        The following two arguments are tensors from a mini-batch of the input_ids
            and attention_mask returned by the BERT tokenizer.
            
            input_ids: a tensor of shape [batch_size, max_length]
            attention_mask: a tensor of shape [batch_size, max_length]
            
            return: the logits of the sentences in the batch tensor of shape [batch_size, 1, 2]
        """
        # see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        # and https://huggingface.co/docs/transformers/main_classes/output
        # for the return value of the forward function of BERT

        ### Your codes go here (30 points) ###
        z = self.bert.forward(input_ids, attention_mask)
        return torch.softmax(self.classifier(z[1]).unsqueeze(1), dim=-1) 
        
        return logits

In [21]:
# make sure you check the availability of GPU when you set the device ID.
device = torch.device(0)

BATCH_SIZE = 8
num_classes = 8
classifier_hidden_size = 128

## if using MAGIC
BERT_PATH = '/data/badassnlp/bert-base-uncased/'
## if using Google Colab, you need to load the bert model after it downloads the model files.
## 

BERT_hidden_size = 768

N_EPOCHS = 1
CLIP = 1

In [22]:
## 'uncased' means all words are lowered-cased before tokenization
## 'base' means the smaller version of BERT (12 layers, 16 heads)
## un-comment one of the following two options.

# if using MAGIC, load from local BERT folder
tokenizer = BertTokenizer.from_pretrained(BERT_PATH, local_files_only=True)
BERT_model = BertModel.from_pretrained(BERT_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False
).to(device)

## if using Colab, load from automatically downloaded files. Downloading can take half a minute
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# BERT_model = BertModel.from_pretrained("bert-base-cased", num_labels = 2, output_attentions = False, output_hidden_states = False).to(device)


## if using MAGIC
print("creating training corpora ... ")
training_corpora = Corpora(train_list)
print("creating training dataset ... ")
training_dataset = TextClassificationDataSet(training_corpora, tokenizer)
print("creating training iterator ... ")
training_iterator = DataLoader(training_dataset, sampler = RandomSampler(training_dataset), batch_size=BATCH_SIZE)

## if using MAGIC
print("creating test corpora ... ")
dev_corpora = Corpora(test_list)
print("creating test dataset ... ")
dev_dataset = TextClassificationDataSet(dev_corpora, tokenizer)
print("creating test iterator ... ")
dev_iterator = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size=BATCH_SIZE)

classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

Some weights of the model checkpoint at /data/badassnlp/bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


creating training corpora ... 


100%|██████████| 5016/5016 [00:00<00:00, 1359347.99it/s]

creating training dataset ... 


creating training iterator ... 
creating test corpora ... 


100%|██████████| 7422/7422 [00:00<00:00, 1260737.25it/s]

creating test dataset ... 


creating test iterator ... 


In [23]:
list(training_iterator)[0][0].shape

torch.Size([8, 210])

In [24]:
list(dev_iterator)[0][0].shape

torch.Size([8, 210])

In [25]:
from torch import optim
optimizer = optim.Adam(classifier.parameters())
criterion = nn.CrossEntropyLoss(reduction = 'sum')

# Global variable that count the number of training epoch
num_epochs_train = 0

from tqdm import tqdm

In [26]:
def train(model, iterator, optimizer, criterion, clip):
    """
        model: an BERTClassifier object
        iterator: a DataLoader object
        optimizer: torch optimizer
        criterion: the crossentropyloss
        clip: to be used with torch.nn.utils.clip_grad_norm_

        return: average loss over the training instances (sentences) in the DataLoader.
    """
    model.train()
    
    epoch_loss = 0
    num_batchs = 0
    total_instances = 0
    global num_epochs_train
    num_epochs_train += 1
    for i, batch in tqdm(enumerate(iterator)):
        num_batchs += 1
        optimizer.zero_grad() 
        tmp = optimizer.state_dict()
        tmp["param_groups"][0]["lr"] = 0.00002/(num_epochs_train)
        optimizer.load_state_dict(tmp)

        ### Your codes go here (10 points) ###

        # Step 1 (5 points): get the tensors from this mini-batch and increase the total_instances
        # make sure tensors are moved to GPU.
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]).to(device)
        
        # Step 2 (5 points): call the forward function of the model and find the output logits
        # then calculate the loss. Then cumulate the epoch_loss
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) # if squeeze here, why unsqueeze previous
        loss.backward()

        # Clips gradient norm of an iterable of parameters.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()
        total_instances += BATCH_SIZE

    return epoch_loss / total_instances

confusion_matrix = []
num_epochs = 0

def evaluate(model, iterator, criterion):
    """
        Find the loss (criterion) of the model over instances in this DataLoader (iterator)
        Same logic as the above train function, except no gradient calculation
        and no update of the parameter. 
        
        return: average loss over the training instances (sentences) in the DataLoader.
    """
    model.eval()

    epoch_loss = 0
    total_instances = 0
    
    confusion_matrix.append(torch.zeros(num_classes,num_classes))
    global num_epochs
    
    for i, batch in tqdm(enumerate(iterator)):
        
        ### Your codes go here (5 points) ###
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]).to(device)
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) # if squeeze here, why unsqueeze previous
        
        epoch_loss += loss.item()
        total_instances += BATCH_SIZE
        

        for i in range(len(y)):
            row = y[i]
            col = torch.argmax((logits.squeeze(1))[i])
            confusion_matrix[num_epochs][row][col] += 1
    num_epochs += 1

    return epoch_loss / total_instances

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [27]:
import time

best_test_loss = float('inf')
training_losses = []
test_losses = []

In [63]:
for epoch in range(N_EPOCHS):  
    print("epoch start: ", epoch)  
    start_time = time.time()
    training_loss = train(classifier, training_iterator, optimizer, criterion, CLIP)
    training_losses.append(training_loss)
    test_loss = evaluate(classifier, dev_iterator, criterion)
    test_losses.append(test_loss)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if test_loss < best_test_loss:
        best_test_loss = test_loss 
        sd = classifier.state_dict()
        for i in sd.keys():
            sd[i] = sd[i].cpu()
        torch.save(sd, 'best_model_8classes_balanced.pt')
        

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s', end='')
    print(f'\tTrain Loss: {training_loss:.3f} | Test Loss: {test_loss:.3f}')

epoch start:  0


627it [02:06,  4.97it/s]
928it [00:44, 21.04it/s]


Epoch: 01 | Time: 2m 50s	Train Loss: 1.794 | Test Loss: 1.890


In [64]:
test_losses

[1.952525275534597, 1.9082042533775856, 1.8899710946555794]

In [65]:
# ['problems with friends', 'ongoing depression','breakup with partner', 'academic pressure', 'job crisis', 'Family Issues','Sleep Problems']
list(confusion_matrix[0])

[tensor([733.,   0.,   0.,  18., 158.,   0.,  22.,   4.]),
 tensor([1114.,    0.,    0.,   61.,  577.,    2.,  117.,    8.]),
 tensor([1.0220e+03, 0.0000e+00, 0.0000e+00, 1.3000e+01, 2.5200e+02, 1.0000e+00,
         4.2000e+01, 3.0000e+00]),
 tensor([262.,   0.,   0., 307., 265.,   0.,  31.,   0.]),
 tensor([536.,   0.,   0.,  84., 810.,   0.,  55.,   4.]),
 tensor([128.,   0.,   0.,   2.,  26.,   0.,   1.,   0.]),
 tensor([65.,  0.,  0.,  5., 37.,  0., 30.,  0.]),
 tensor([ 10.,   0.,   0.,   1.,   0.,   0.,   1., 615.])]

In [68]:
import numpy as np
sentence = "cought a cold"
es = torch.tensor([tokenizer.encode(sentence)])
msk = torch.ones(len(es[0])).unsqueeze(0)
es = es.to(device)
msk = msk.to(device)
logits = classifier.forward(es,msk)
print(logits)
kept[torch.argmax(logits)]

tensor([[[2.9914e-03, 2.4935e-04, 1.3663e-04, 1.2401e-04, 3.4031e-04,
          1.9573e-03, 1.9695e-04, 9.9400e-01]]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


'illness'

In [ ]:
# current issue:
# Since this is patient note, different from conv data, it uses 'patients,' 'he', 'she'
# , rather than using I or my. The word I will force the illness utterance into other catorgories
# We need to change the data